In [ ]:
%pip install requests beautifulsoup4 selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json

def scrape_facebook_without_login(profile_url, num_posts=5):
    """Скрапінг Facebook-постів без входу, закриваючи попап авторизації та збереження в JSONL."""

    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # Запуск в headless-режимі (без інтерфейсу)
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
    
    driver = webdriver.Chrome(options=chrome_options)

    # Відкриваємо файл для збереження постів
    with open('facebook_posts.jsonl', 'w') as file:
        try:
            driver.get(profile_url)
            time.sleep(5)  # Чекаємо, щоб сторінка завантажилась

            # 🔴 Спроба закрити попап входу
            try:
                close_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "//div[contains(@aria-label, 'Закрити') or contains(@aria-label, 'Close')]"))
                )
                close_button.click()
                print("Попап авторизації закрито.")
                time.sleep(2)  # Дати сторінці оновитися після закриття
            except Exception:
                print("Попап не з’явився або його не вдалося закрити.")

            # 🔵 Прокручуємо сторінку вниз для завантаження більше постів
            for _ in range(3):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(3)

            # Отримуємо HTML-код сторінки
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            # 🔵 Знаходимо блоки з постами
            post_containers = soup.find_all('div', class_='x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z')

            if not post_containers:
                print("❌ Постів не знайдено. Можливо, потрібно увійти в акаунт.")
                return

            # 🔵 Обробка постів і запис у файл
            for container in post_containers[:num_posts]:
                try:
                    # 🔴 Спроба натиснути "Більше", щоб розкрити текст
                    try:
                        read_more_button = container.find_element(By.XPATH, ".//div[contains(text(), 'Більше')]")
                        driver.execute_script("arguments[0].click();", read_more_button)
                        time.sleep(1)
                    except Exception:
                        pass  # Якщо кнопки немає, ігноруємо помилку

                    # 🔵 Отримання тексту поста
                    description = ""
                    description_element = container.find('div', {'data-ad-comet-preview': 'message'})
                    if description_element:
                        desc_divs = description_element.find_all('div', dir='auto')
                        for div in desc_divs:
                            description += div.get_text(separator='\n', strip=True) + '\n'
                        description = description.strip()
                    else:
                        description = "No description found"

                    # 🔵 Отримання посилань на зображення
                    image_links = []
                    img_elements = container.find_all('img')
                    for img in img_elements:
                        src = img.get('src')
                        if src:
                            image_links.append(src)

                    # Створення словника для збереження поста
                    post_data = {
                        "description": description,
                        "image_links": image_links
                    }

                    # Збереження поста в JSONL файл
                    json.dump(post_data, file)
                    file.write('\n')  # Кожен об'єкт на новому рядку
                    print(f"📌 Опис: {description}\n🖼️ Зображення: {image_links}\n---")

                except Exception as e:
                    print(f"❌ Помилка обробки поста: {e}")

        finally:
            driver.quit()

# ✅ Виклик функції
profile_url = "https://www.facebook.com/providentrealestateuz"
scrape_facebook_without_login(profile_url)


Попап авторизації закрито.
📌 Опис: Продаж квартири студії у новобудові
Загальна площа - 31 м2
Поверх - 4 з 5 (ліфт)
Опалення - індивідуальне електричне (світло не вимикають )
Тепла підлога по всій квартирі …
Більше
🖼️ Зображення: ['https://scontent-waw2-1.xx.fbcdn.net/v/t39.30808-6/485012066_1068979898597816_962385903184677010_n.jpg?stp=dst-jpg_s600x600_tt6&_nc_cat=110&ccb=1-7&_nc_sid=833d8c&_nc_ohc=G2u8W2XpSbEQ7kNvgERPzSm&_nc_oc=AdnFH3D--cZjjfFUg-GcY624XTGIyfQGHjc-tRXha-UTHVK0UHWReTuGm-YuAimz6rc&_nc_zt=23&_nc_ht=scontent-waw2-1.xx&_nc_gid=I42S0AxRqEtufTh7B0_NDQ&oh=00_AYGxAfcR7e-Lj2Bc21Jp8OKrDufkAVkBOdHPGEC5ACBmrA&oe=67E8621B', 'https://scontent-waw2-2.xx.fbcdn.net/v/t39.30808-6/484670277_1068979905264482_8251240872386242879_n.jpg?stp=dst-jpg_p480x480_tt6&_nc_cat=105&ccb=1-7&_nc_sid=833d8c&_nc_ohc=6CEIKqDtQuMQ7kNvgFoi5HA&_nc_oc=AdkFx601XcUtJrjE38kZRtm8zRP6v81h5schxpLnBg1ZvSB-qYazxvb_gdrwhyv1_J4&_nc_zt=23&_nc_ht=scontent-waw2-2.xx&_nc_gid=I42S0AxRqEtufTh7B0_NDQ&oh=00_AYFs8c6L0A7uvOhHHJi